# Explore here

In [60]:
import pandas as pd
from utils import db_connect
from app import query
import matplotlib.pyplot as plt 
import seaborn as sns
import json

In [61]:
engine = db_connect()

# Consultar la tabla en la base de datos y crear un DataFrame
total_data = pd.read_sql(query, engine)

pd.set_option('display.max_columns', None)  # muestra todas las columnas del dataframe
total_data.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,title,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [62]:
total_data.to_csv('/workspaces/K-NearestNeighbors/data/raw/5000-films.csv', index=False)

In [63]:
total_data.shape

(4809, 24)

In [64]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [65]:
total_data.drop(['homepage', 'id', 'tagline'], axis=1, inplace=True)

In [66]:
total_data["genres"] = total_data["genres"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)

# .apply(lambda x: ...): Aplica una función a cada elemento de la columna "genres".
# json.loads(x): Convierte la cadena JSON en una lista de diccionarios en Python.
# [item["name"] for item in ...]: comprensión de lista para extraer el valor de la clave "name" de cada diccionario en la lista.
# if pd.notna(x) else None: maneja los valores nulos en la columna convirtiéndolos en None.

In [67]:
total_data["overview"] = total_data["overview"].apply(lambda x: [x])

In [68]:
total_data["keywords"] = total_data["keywords"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)
total_data["spoken_languages"] = total_data["spoken_languages"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)
total_data["production_companies"] = total_data["production_companies"].apply(lambda x: [item["name"] for item in json.loads(x)][:3] if pd.notna(x) else None)
total_data["production_countries"] = total_data["production_countries"].apply(lambda x: [item["name"] for item in json.loads(x)][:3] if pd.notna(x) else None)
total_data["cast"] = total_data["cast"].apply(lambda x: [item["name"] for item in json.loads(x)][:3] if pd.notna(x) else None)

In [69]:
total_data["crew"] = total_data["crew"].apply(lambda x: " ".join([crew_member['name'] for crew_member in json.loads(x) if crew_member['job'] == 'Director']))

# json.loads(x): Carga los datos JSON de la cadena x en una lista de Python de diccionarios.
# for crew_member in json.loads(x): Recorre cada miembro del equipo en los datos JSON.
# if crew_member['job'] == 'Director': Filtra solo los miembros del equipo cuyo trabajo es "Director".
# crew_member['name']: Extrae el nombre de cada miembro del equipo.
# " ".join(...): Une los nombres de los directores en una cadena, separados por espacios.

In [70]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   keywords              4809 non-null   object 
 3   original_language     4809 non-null   object 
 4   original_title        4809 non-null   object 
 5   overview              4809 non-null   object 
 6   popularity            4809 non-null   float64
 7   production_companies  4809 non-null   object 
 8   production_countries  4809 non-null   object 
 9   release_date          4808 non-null   object 
 10  revenue               4809 non-null   int64  
 11  runtime               4807 non-null   float64
 12  spoken_languages      4809 non-null   object 
 13  status                4809 non-null   object 
 14  title                 4809 non-null   object 
 15  vote_average         

In [71]:
total_data = total_data.loc[:, ~total_data.columns.duplicated()]

In [72]:
total_data.isnull().any()

budget                  False
genres                  False
keywords                False
original_language       False
original_title          False
overview                False
popularity              False
production_companies    False
production_countries    False
release_date             True
revenue                 False
runtime                  True
spoken_languages        False
status                  False
title                   False
vote_average            False
vote_count              False
movie_id                False
cast                    False
crew                    False
dtype: bool

In [73]:
total_data.head(2)

,budget,genres,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"[In the 22nd century, a paraplegic Marine is d...",150.437577,"[Ingenious Film Partners, Twentieth Century Fo...","[United States of America, United Kingdom]",2009-12-10,2787965087,162.0,"[English, Español]",Released,Avatar,7.2,11800,19995,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,300000000,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...",139.082615,"[Walt Disney Pictures, Jerry Bruckheimer Films...",[United States of America],2007-05-19,961000000,169.0,[English],Released,Pirates of the Caribbean: At World's End,6.9,4500,285,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski


In [74]:
total_data['description'] = total_data['genres'] + total_data['keywords'] + total_data['overview'] + total_data['production_companies'] + total_data['production_countries'] + total_data['spoken_languages'] + total_data['cast']

total_data.drop(['genres', 'keywords', 'overview', 'production_companies', 'production_countries', 'spoken_languages', 'cast'], axis=1, inplace=True)

In [75]:
total_data.drop(['original_language', 'original_title', 'movie_id'], axis=1, inplace=True)
total_data.head(2)

,budget,popularity,release_date,revenue,runtime,status,title,vote_average,vote_count,crew,description
0,237000000,150.437577,2009-12-10,2787965087,162.0,Released,Avatar,7.2,11800,James Cameron,"[Action, Adventure, Fantasy, Science Fiction, ..."
1,300000000,139.082615,2007-05-19,961000000,169.0,Released,Pirates of the Caribbean: At World's End,6.9,4500,Gore Verbinski,"[Adventure, Fantasy, Action, ocean, drug abuse..."


In [76]:
total_data['description'] = total_data['description'].apply(lambda x: " ".join([item for item in x if isinstance(item, str)]) if x is not None else "")

In [78]:
total_data["description"] = total_data["description"].str.strip().str.lower()

In [79]:
total_data.iloc[20].description

"action comedy science fiction time travel time machine alien fictional government agency seeing the future changing history agents j (will smith) and k (tommy lee jones) are back...in time. j has seen some inexplicable things in his 15 years with the men in black, but nothing, not even aliens, perplexes him as much as his wry, reticent partner. but when k's life and the fate of the planet are put at stake, agent j will have to travel back in time to put things right. j discovers that there are secrets to the universe that k never told him - secrets that will reveal themselves as he teams up with the young agent k (josh brolin) to save his partner, the agency, and the future of humankind. amblin entertainment media magik entertainment imagenation abu dhabi fz united states of america english will smith tommy lee jones josh brolin"

In [81]:
import re

def remove_punctuation(text):
    # Define una expresión regular para encontrar signos de puntuación
    punctuation_pattern = r'[^\w\s]'
    # Reemplaza los signos de puntuación por una cadena vacía
    text_without_punctuation = re.sub(punctuation_pattern, '', str(text))
    return text_without_punctuation

# Aplicar la función remove_punctuation a toda la columna 'description'
total_data['description'] = total_data['description'].apply(remove_punctuation)

# Mostrar la descripción de la primera fila después de eliminar la puntuación
total_data.iloc[0].description

'action adventure fantasy science fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d in the 22nd century a paraplegic marine is dispatched to the moon pandora on a unique mission but becomes torn between following orders and protecting an alien civilization ingenious film partners twentieth century fox film corporation dune entertainment united states of america united kingdom english español sam worthington zoe saldana sigourney weaver'